# Educational Tutor AI Assistant: Code Walkthrough

In this walkthrough, we'll go through the code to build an Educational Tutor AI Assistant using Google's Gemini language model. The assistant will include features like generating quizzes, creating flashcards, providing grammar corrections, and offering project guidance.

**Environment Setup**

**Installing Required Packages**

First, we need to install the necessary Python packages:

In [2]:
# Install Google Generative AI client and litellm
!pip install --quiet google-generativeai litellm

**google-generativeai:** The client library for accessing Google's Generative AI models.

**litellm:** A lightweight library that provides a unified interface for different language models.

## Importing Libraries

Next, we import the required libraries:

In [3]:
import os
import json
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from pydantic import BaseModel, Field
from typing import List

**crewai:** A library for managing AI agents and tasks.

**pydantic:** Used for data validation and settings management using Python type annotations.

**typing:** Provides support for type hints.

## API Key Configuration

We need to set up the API key for accessing the Gemini model:

In [24]:
# Set up the Gemini API key
os.environ["GEMINI_API_KEY"] = "GEMINI_API_KEY"  # Replace with your actual API key

**Security Note:** Replace "YOUR_GEMINI_API_KEY" with your actual API key.

Ensure you do not expose your API key in shared code or public repositories.

## Initializing the Language Model

We initialize the LLM class from crewai with the Gemini model:

In [5]:
# Initialize the LLM with the Gemini model using the correct model name
gemini_llm = LLM(model="gemini/gemini-1.5-pro", temperature=0)

**Parameters:**

**model:** Specifies the language model to use. The prefix gemini/ indicates the provider.

**temperature:** Controls the randomness of the output. A value of 0 makes the output more deterministic.

## Defining the Knowledge Base

Create a knowledge base that the agents can reference:

In [6]:
# Define the knowledge base
educational_content = """
This knowledge base contains information on programming concepts, data analytics, and language learning materials.
"""

tutor_knowledge = StringKnowledgeSource(
    content=educational_content
)

**StringKnowledgeSource:** Allows us to provide knowledge content as a string.

This content can be expanded to include detailed learning materials.

## Output Directory Setup

We define an output directory to store generated outputs:

In [7]:
# Output directory
output_dir = "./educational-tutor-output"
os.makedirs(output_dir, exist_ok=True)

## Data Models Definition

We define data models using pydantic for the outputs we expect from our agents.

In [8]:
class QuizQuestion(BaseModel):
    question: str
    options: List[str]
    correct_answer: int  # Index of the correct answer in options

class Quiz(BaseModel):
    topic: str
    questions: List[QuizQuestion]

**QuizQuestion:** Represents a single quiz question.

**Quiz:** Represents a collection of quiz questions on a topic.

## Flashcard Data Models

In [9]:
class Flashcard(BaseModel):
    front: str = Field(..., title="Term or question")
    back: str = Field(..., title="Definition or answer")

class Flashcards(BaseModel):
    topic: str
    cards: List[Flashcard]

**Flashcard:** Represents a single flashcard with a front and back.

**Flashcards:** Represents a set of flashcards on a topic.


## Grammar Correction Data Model

In [10]:
class CorrectionFeedback(BaseModel):
    original_text: str
    corrected_text: str
    feedback: str

**CorrectionFeedback:** Contains the original text, corrected text, and feedback.


## Project Guidance Data Model


In [11]:
class ProjectGuidance(BaseModel):
    project_topic: str
    resources: List[str]
    steps: List[str]
    tips: List[str]

**ProjectGuidance:** Provides guidance for a project, including resources, steps, and tips.


## Defining Agents

Agents are responsible for performing specific tasks. We define an agent for each functionality.

### Quiz Generator Agent


In [12]:
quiz_generator_agent = Agent(
    role="Quiz Generator Agent",
    goal="Create interactive quizzes on a given topic to help learners test their knowledge.",
    backstory="The agent generates quizzes that are engaging and educational.",
    llm=gemini_llm,
    verbose=True,
)

### Flashcards Creator Agent

In [13]:
flashcards_creator_agent = Agent(
    role="Flashcards Creator Agent",
    goal="Create flashcards to help learners memorize key concepts.",
    backstory="The agent summarizes information into digestible flashcards.",
    llm=gemini_llm,
    verbose=True,
)

### Grammar Correction Agent

In [14]:
grammar_correction_agent = Agent(
    role="Grammar Correction Agent",
    goal="Provide real-time grammar correction and feedback for learners.",
    backstory="The agent helps learners improve their language skills by correcting mistakes.",
    llm=gemini_llm,
    verbose=True,
)

### Project Guidance Agent

In [ ]:
project_guidance_agent = Agent(
    role="Project Guidance Agent",
    goal="Provide step-by-step guidance on projects in topics like Python, SQL, or Tableau.",
    backstory="The agent assists learners in completing projects by breaking down tasks.",
    llm=gemini_llm,
    verbose=True,
)

# Defining Tasks

Tasks specify what each agent should do and how to handle the outputs.

### Quiz Generation Task

In [ ]:
quiz_generation_task = Task(
    description="Generate a quiz with multiple-choice questions on the topic '{topic}'. Each question should have 4 options.",
    expected_output="A JSON object containing the quiz questions and answers.",
    output_json=Quiz,
    output_file=os.path.join(output_dir, "quiz.json"),
    agent=quiz_generator_agent
)

### Flashcards Creation Task


In [17]:
flashcards_creation_task = Task(
    description="Create a set of flashcards for the topic '{topic}'. Each flashcard should focus on a key concept or term.",
    expected_output="A JSON object containing the flashcards.",
    output_json=Flashcards,
    output_file=os.path.join(output_dir, "flashcards.json"),
    agent=flashcards_creator_agent
)

### Grammar Correction Task


In [18]:
grammar_correction_task = Task(
    description="Correct the grammar of the following text and provide feedback.\n\nText: '{text}'",
    expected_output="A JSON object containing the corrected text and feedback.",
    output_json=CorrectionFeedback,
    output_file=os.path.join(output_dir, "grammar_correction.json"),
    agent=grammar_correction_agent
)

### Project Guidance Task


In [19]:
project_guidance_task = Task(
    description="Provide guidance for a project on '{project_topic}'. Include resources, steps, and tips.",
    expected_output="A JSON object containing the project guidance.",
    output_json=ProjectGuidance,
    output_file=os.path.join(output_dir, "project_guidance.json"),
    agent=project_guidance_agent
)

# Creating and Running the Crew

We assemble the agents and tasks into a crew and run it.

In [25]:
import time

crew_results = educational_tutor_crew.kickoff(
    inputs={
        "topic": "Introduction to Python Programming",
        "text": "def my_function()\n print('Hello World')",
        "project_topic": "Building a Simple Calculator in Python",
    }
)

time.sleep(5)  # Introduce a 5-second delay between requests

# Create the Crew and run
educational_tutor_crew = Crew(
    agents=[
        quiz_generator_agent,
        flashcards_creator_agent,
        grammar_correction_agent,
        project_guidance_agent,
    ],
    tasks=[
        quiz_generation_task,
        flashcards_creation_task,
        grammar_correction_task,
        project_guidance_task,
    ],
    process=Process.sequential,
    knowledge_sources=[tutor_knowledge]
)

crew_results = educational_tutor_crew.kickoff(
    inputs={
        "topic": "Introduction to Python Programming",
        "text": "def my_function()\n print('Hello World')",
        "project_topic": "Building a Simple Calculator in Python",
    }
)

# Agent: Quiz Generator Agent
## Task: Generate a quiz with multiple-choice questions on the topic 'Introduction to Python Programming'. Each question should have 4 options.


# Agent: Quiz Generator Agent
## Final Answer: 
{
  "topic": "Introduction to Python Programming",
  "questions": [
    {
      "question": "What is the correct way to print 'Hello, World!' in Python?",
      "options": [
        "print('Hello, World!')",
        "console.log('Hello, World!')",
        "System.out.println('Hello, World!')",
        "echo('Hello, World!')"
      ],
      "correct_answer": 0
    },
    {
      "question": "Which data type is used to store a sequence of characters in Python?",
      "options": [
        "string",
        "char",
        "array",
        "text"
      ],
      "correct_answer": 0
    },
    {
      "question": "What is the output of the following code?\n\nx = 5\ny = 2\nprint(x // y)",
      "options": [
        "2.5",
        "2",
        "2.0",
        "3"
      ],
 

ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}



# Agent: Quiz Generator Agent
## Task: Generate a quiz with multiple-choice questions on the topic 'Introduction to Python Programming'. Each question should have 4 options.


LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}


# Accessing the Outputs
After execution, the outputs are saved in the specified output directory.

###Reading the Generated Quiz

In [29]:
with open(os.path.join(output_dir, "quiz.json"), "r") as f:
    quiz = json.load(f)

print("Quiz Topic:", quiz['topic'])
for idx, question in enumerate(quiz['questions'], 1):
    print(f"\nQuestion {idx}: {question['question']}")
    for opt_idx, option in enumerate(question['options'], 1):
        print(f"{opt_idx}. {option}")
    print(f"Correct Answer: Option {question['correct_answer'] + 1}")

Quiz Topic: Introduction to Python Programming

Question 1: What is the correct way to print 'Hello, World!' in Python?
1. print('Hello, World!')
2. console.log('Hello, World!')
3. System.out.println('Hello, World!')
4. echo('Hello, World!')
Correct Answer: Option 1

Question 2: Which data type is used to store a sequence of characters in Python?
1. string
2. char
3. array
4. text
Correct Answer: Option 1

Question 3: What is the output of the following code?

x = 5
y = 2
print(x // y)
1. 2.5
2. 2
3. 2.0
4. 3
Correct Answer: Option 2

Question 4: Which keyword is used to define a function in Python?
1. def
2. function
3. define
4. fun
Correct Answer: Option 1

Question 5: What does the `len()` function do?
1. Returns the length of an object
2. Returns the data type of an object
3. Returns the value of an object
4. Prints the object to the console
Correct Answer: Option 1

Question 6: Which of the following is NOT a valid variable name in Python?
1. my_variable
2. 123variable
3. _variab

### Reading the Created Flashcards


In [26]:
with open(os.path.join(output_dir, "flashcards.json"), "r") as f:
    flashcards = json.load(f)

print("Flashcards Topic:", flashcards['topic'])
for idx, card in enumerate(flashcards['cards'], 1):
    print(f"\nFlashcard {idx}:")
    print("Front:", card['front'])
    print("Back:", card['back'])

Flashcards Topic: Introduction to Python Programming

Flashcard 1:
Front: What is the correct way to print 'Hello, World!' in Python?
Back: print('Hello, World!')

Flashcard 2:
Front: Which data type is used to store a sequence of characters in Python?
Back: string

Flashcard 3:
Front: What is the output of the following code?

x = 5
y = 2
print(x // y)
Back: 2

Flashcard 4:
Front: Which keyword is used to define a function in Python?
Back: def

Flashcard 5:
Front: What does the `len()` function do?
Back: Returns the length of an object

Flashcard 6:
Front: Which of the following is NOT a valid variable name in Python?
Back: 123variable

Flashcard 7:
Front: What is the purpose of the `if` statement?
Back: To execute a block of code conditionally

Flashcard 8:
Front: How do you add a comment in Python?
Back: # This is a comment

Flashcard 9:
Front: What is the result of `True and False`?
Back: False

Flashcard 10:
Front: Which operator is used for exponentiation in Python?
Back: **


### Reading the Grammar Correction Feedback


In [27]:
with open(os.path.join(output_dir, "grammar_correction.json"), "r") as f:
    correction = json.load(f)

print("Original Text:\n", correction['original_text'])
print("\nCorrected Text:\n", correction['corrected_text'])
print("\nFeedback:\n", correction['feedback'])

Original Text:
 def my_function()
 print('Hello World')

Corrected Text:
 def my_function():
    print('Hello World')

Feedback:
 In Python, indentation is crucial for defining code blocks within functions. Your original code was missing the necessary indentation for the `print` statement inside the `my_function` definition.  The corrected version includes four spaces to indent the `print` statement, indicating that it belongs to the function's body.  This ensures that when you call `my_function()`, 'Hello World' will be printed. Remember, consistent indentation is essential for Python code to execute correctly.  Inconsistent indentation will lead to `IndentationError`.


### Reading the Project Guidance


In [28]:
with open(os.path.join(output_dir, "project_guidance.json"), "r") as f:
    guidance = json.load(f)

print("Project Topic:", guidance['project_topic'])
print("\nResources:")
for resource in guidance['resources']:
    print("-", resource)
print("\nSteps:")
for step in guidance['steps']:
    print("-", step)
print("\nTips:")
for tip in guidance['tips']:
    print("-", tip)

Project Topic: Building a Simple Calculator in Python

Resources:
- https://docs.python.org/3/tutorial/index.html
- https://www.w3schools.com/python/
- https://realpython.com/

Steps:
- **1. Set up your environment:** Install Python and choose a suitable IDE or text editor.
- **2. Design the user interface:** Decide whether you'll use a basic text-based interface or a graphical user interface (GUI) library like Tkinter.
- **3. Implement basic arithmetic operations:** Create functions for addition, subtraction, multiplication, and division. Handle potential errors like division by zero.
- **4. Get user input:**  If using a text-based interface, use the `input()` function to get numbers and the desired operation from the user. For a GUI, use entry widgets and buttons.
- **5. Perform calculations:** Based on the user's input, call the appropriate arithmetic function.
- **6. Display the result:** Print the calculated result to the console or display it in a GUI element.
- **7. (Optional) A